In [ ]:
device_id = 0
device = 'cuda:' + str(device_id)
#device = 'cpu'

import time
import os
import torch

log_dir = os.path.join('./log/question_classifier', time.strftime('%Y%m%d_%H%M%S', time.localtime(time.time())))
print(log_dir)

In [ ]:
from pkg.parse import *
from pkg.words import *
from problems import *

problems = [P1_1_1, P1_1_2, P1_1_3, P1_1_4, P1_1_5, P1_1_6, P1_1_7, P1_1_8, P1_1_9, P1_1_10, P1_1_11, P1_1_12, 
            P1_2_1, P1_2_2, P1_3_1, P1_4_1, 
            P2_1_1, P2_2_2, P2_3_1, 
            P3_1_1, P3_2_1, P3_2_2, P3_3_1, 
            P4_1_1, P4_2_1, P4_2_2, P4_3_1, 
            P5_1_1, P5_2_1, P5_3_1,
            P6_1_1, P6_3_1, P6_4_1,
            P7_1_1, P7_1_2, P7_3_1,
            P8_1_1, P8_2_1, P8_3_1, 
            P9_1_1, P9_2_1, P9_2_2, P9_3_1, P9_3_2]

In [ ]:
from pkg.trainer_question_classifier import Hyper as hp

# dataset
hp.add_bos = False
hp.add_eos = False

hp.batch_size = None
hp.batch_type = 'normal'
hp.ds_batch_size = 128

# train
hp.num_workers = 4

hp.steps_log = 10
hp.steps_eval = 50
hp.steps_save = 10000

hp.weight_decay = 0.000001
hp.initial_lr = 0.0001
hp.final_lr = 0.00001
hp.lr_decay_factor = 0.99
hp.lr_patience = 300
hp.ema = 0.99
hp.grad_norm_max = 10.0

hp.adam_alpha = 2e-4
hp.adam_betas = (0.5, 0.9)
hp.adam_eps = 1e-6

hp.vocab_size = 512
hp.add_space_token = False

hp.num_problems = len(problems)
hp.problems = problems


In [ ]:
from pkg.dataset import ProblemDataset, QuestionDataset, read_questions, write_questions

dir_question = 'data/question'
_prefix = 'question_'
_ids = [0] #, 1, 2, 3, 4, 5, 6, 7, 8, 9]
path_questions = [os.path.join(dir_question, _prefix + str(i) + '.txt') for i in _ids]
metas = [read_questions(path) for path in path_questions]
meta = []
for m in metas:
    meta += m
len_train = int(len(meta) * 0.8)
meta_train = meta[:len_train]
meta_val = meta[len_train:]
print(len(meta), len(meta_train), len(meta_val))

In [ ]:
from pkg.vocab import Vocab, CharVocab, SPVocab
dir_token = 'tokenization'
filename = 'prob'
filename += '_' + str(hp.vocab_size)
if hp.add_space_token:
    filename += '_'
filename += '.model'
path_model = os.path.join(dir_token, filename)
print(path_model)
vocab = SPVocab(path_model)
print(vocab.vocab_size())

In [ ]:
start = time.time()
dsTrain = QuestionDataset(meta_train, vocab, batch_size=hp.ds_batch_size)
dsVal = QuestionDataset(meta_val, vocab, batch_size=hp.ds_batch_size)
print(time.time() - start)

In [ ]:
from pkg.models.model import QuestionClassifier
from pkg.models.config import Config
from pkg.models.extractor import AverageExtractor, RNNExtractor
from pkg.models.encoders_conv import ConvEncoder, HighwayEncoder
from pkg.models.embedding import Embed, Regressor

cfg = QuestionClassifier.default_config()
cfg.text_embed.num_symbols = hp.vocab_size
cfg.regressor.num_symbols = hp.num_problems
model = cfg.create_object().to(device)

In [ ]:
from pkg.trainer_question_classifier import Trainer_QuestionClassifier

trainer = Trainer_QuestionClassifier(model, dsTrain, dsVal, hp=hp, log_dir=log_dir)

In [ ]:
trainer.train()